In [2]:
## 🚀 Setup and Helper Functions

import json
from datetime import datetime, timedelta
import glob
import os

# --- Helper function for half-second target calculation ---

def get_half_second_target(ts_datetime):
    """
    Calculates the nearest .000 or .500 target time for a given datetime.
    """
    milliseconds = ts_datetime.microsecond // 1000
    remainder = milliseconds % 500

    if remainder < 250:
        milliseconds -= remainder
    else:
        milliseconds += (500 - remainder)

    if milliseconds == 1000:
        # Handle overflow to the next second
        target = ts_datetime + timedelta(seconds=1)
        target = target.replace(microsecond=0)
    else:
        target = ts_datetime.replace(microsecond=milliseconds * 1000)

    return target


def update_timestamps(data):
    """
    Updates timestamps using a list-based, multi-pass process to prevent data loss,
    prioritize half-second rounding, and ensure strict chronological order with
    gameMoves having priority.
    """

    # 1. Data Structuring and Pre-Sorting
    all_moves_list = []

    # Collect gameMoves
    for ts_str, value in data['gameMoves'].items():
        all_moves_list.append({
            'ts_dt': datetime.fromisoformat(ts_str.replace('Z', '+00:00')),
            'type': 'gameMoves',
            'value': value,
            'final_ts_dt': None # Placeholder for final assigned time
        })

    # Collect doSomethingTimestamps
    for ts_str, value in data['doSomethingTimestamps'].items():
        all_moves_list.append({
            'ts_dt': datetime.fromisoformat(ts_str.replace('Z', '+00:00')),
            'type': 'doSomethingTimestamps',
            'value': value,
            'final_ts_dt': None
        })

    # Define the custom sort key for priority:
    # Primary Sort Key: Original timestamp (must be chronological)
    # Secondary Sort Key: Type (gameMoves: 0, doSomethingTimestamps: 1)
    def sort_key(move):
        priority = 0 if move['type'] == 'gameMoves' else 1
        return (move['ts_dt'], priority)

    # Sort the list to process them in chronological order, with gameMoves breaking ties
    all_moves_list.sort(key=sort_key)

    # 2. Assignment Pass - Assign Target Times

    # Calculate the target half-second for every move
    for move in all_moves_list:
        move['target_ts_dt'] = get_half_second_target(move['ts_dt'])

    # 3. Collision Resolution Pass

    final_timestamps_set = set() # Tracks all assigned times (for checking slot availability)
    previous_final_ts = None      # Tracks the previously assigned time (for chronological guarantee)

    for move in all_moves_list:
        target_ts = move['target_ts_dt']

        # A. Determine the preferred time based on priority/target slot availability
        preferred_ts = target_ts

        # If the target half-second is already taken, apply the 100ms shift rule
        if target_ts in final_timestamps_set:
            # Target slot is taken (by a move processed earlier, higher priority, or earlier time).
            # The rule is: "If the target half-second is taken, move to target + 100ms"
            preferred_ts = target_ts + timedelta(milliseconds=100)

        # B. Chronological uniqueness check (The strongest rule: must be strictly increasing)
        final_ts = preferred_ts
        if previous_final_ts and final_ts <= previous_final_ts:
            # If the preferred time violates chronological order, push it past the previous move
            # by the minimal 100ms step.
            final_ts = previous_final_ts + timedelta(milliseconds=100)

        # C. Final assignment and update trackers
        move['final_ts_dt'] = final_ts
        final_timestamps_set.add(final_ts)
        previous_final_ts = final_ts

    # 4. Final Rebuild of Dictionaries

    new_game_moves = {}
    new_do_something_timestamps = {}

    for move in all_moves_list:
        # Final output uses string representation of the resolved datetime object
        final_ts_str = move['final_ts_dt'].isoformat().replace('+00:00', 'Z')

        if move['type'] == 'gameMoves':
            new_game_moves[final_ts_str] = move['value']
        else: # doSomethingTimestamps
            new_do_something_timestamps[final_ts_str] = move['value']

    data['gameMoves'] = new_game_moves
    data['doSomethingTimestamps'] = new_do_something_timestamps

    return data

# --- Main processing function (remains the same) ---

def process_files(file_list, output_prefix='_updated'):
    """
    Processes a list of JSON files to update timestamps and saves the
    result to a new file with the specified prefix.
    """
    print(f"--- Starting file processing for {len(file_list)} files ---")

    for input_filepath in file_list:
        try:
            # 1. Determine the output filename
            directory = os.path.dirname(input_filepath)
            filename_with_ext = os.path.basename(input_filepath)
            name, ext = os.path.splitext(filename_with_ext)

            # Construct the new filename: insert '_updated' before the final number
            last_underscore_index = name.rfind('_')
            if last_underscore_index != -1 and name[last_underscore_index+1:].isdigit():
                base_name = name[:last_underscore_index]
                suffix = name[last_underscore_index:]
                new_name = f"{base_name}{output_prefix}{suffix}"
            else:
                new_name = f"{name}{output_prefix}"

            output_filepath = os.path.join(directory, f"{new_name}{ext}")

            print(f"\nProcessing: {input_filepath}")

            # 2. Load the data
            with open(input_filepath, 'r') as f:
                data = json.load(f)

            # Check original lengths before update
            original_gm_len = len(data['gameMoves'])
            original_dst_len = len(data['doSomethingTimestamps'])
            total_original = original_gm_len + original_dst_len

            # 3. Update the timestamps
            updated_data = update_timestamps(data)

            # Check final lengths after update
            final_gm_len = len(updated_data['gameMoves'])
            final_dst_len = len(updated_data['doSomethingTimestamps'])
            total_final = final_gm_len + final_dst_len

            if total_original != total_final:
                # The move from a dictionary-based intermediate step to a list-based step
                # should now prevent data loss, so this check should pass.
                raise ValueError(
                    f"DATA MISMATCH: Original total count ({total_original}) "
                    f"does not match final total count ({total_final})."
                )

            # 4. Save the new data
            with open(output_filepath, 'w') as f:
                json.dump(updated_data, f, indent=4)

            print(f"Output to: {output_filepath}")
            print(f"Successfully updated and saved. Total moves maintained: {total_final}")

        except Exception as e:
            print(f"!!! FATAL ERROR processing {input_filepath}: {e}")

# --- Execution Cell ---

# Define the pattern to find all relevant JSON files.
file_pattern = '../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)/*.json'

# Get the list of all files to process
all_files_to_process = glob.glob(file_pattern)

# Execute the main function
process_files(all_files_to_process)

--- Starting file processing for 11 files ---

Processing: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_1.json
Output to: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_updated_1.json
Successfully updated and saved. Total moves maintained: 472

Processing: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_10.json
Output to: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_updated_10.json
Successfully updated and saved. Total moves maintained: 640

Processing: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_2.json
Output to: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1